This script generates the calibration curve plots (Figure 2), based on the base models.

In [40]:
import numpy as np
import pandas as pd
from scipy import stats
import pickle
from collections import Counter
from sklearn.metrics import precision_recall_curve, roc_auc_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from scipy.stats.mstats import gmean
import math
import datetime as dt
import matplotlib
font = {'size': 24}
matplotlib.rc('font', **font)
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [8]:
df_new = pd.read_stata("df_new.dta")
predictors = pickle.load(open("predictors.p", "rb"))

In [9]:
len(predictors)

294

In [10]:
def calc_cw(y):
    # Calculate the weight of each letter grade to be used in the modeling fitting procedure: the weight is inversely proportional to the square root of the frequency of the letter grade in the training sample
    cw = Counter(y)
    class_weight = {k:np.sqrt(cw.most_common()[0][-1]/v, dtype=np.float32) for k,v in cw.items()}
    return class_weight # The output is a dictionary mapping letter grade to the corresponding weight

In [11]:
train_df = df_new[df_new.valid == 0]
test_df = df_new[df_new.valid == 1]
print(train_df.shape[0], test_df.shape[0])

323182 62618


In [12]:
optimal_d = 16
optimal_n = 120
optimal_nf = 12
rf = RandomForestClassifier(n_estimators=optimal_n, criterion="entropy",
                            max_depth=optimal_d,
                            random_state=0, n_jobs=-1, max_features=optimal_nf,
                            class_weight = calc_cw(train_df.grad_6years))
rf.fit(train_df.loc[:,predictors], train_df.grad_6years)

RandomForestClassifier(bootstrap=True,
            class_weight={0.0: 1.0, 1.0: 1.3931639}, criterion='entropy',
            max_depth=16, max_features=12, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=120, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [13]:
y_test_pred_rf = rf.predict_proba(test_df.loc[:,predictors])[:,1]
y_train_pred_rf = rf.predict_proba(train_df.loc[:,predictors])[:,1]
pickle.dump(y_test_pred_rf, open("y_test_pred_rf.p", "wb"))
pickle.dump(list(test_df.grad_6years), open("y_test.p", "wb"))
pickle.dump(y_train_pred_rf, open("y_train_pred_rf.p", "wb"))
pickle.dump(list(train_df.grad_6years), open("y_train.p", "wb"))

In [14]:
sr = sum(train_df.grad_6years)/train_df.shape[0]
n = int(train_df.shape[0] - train_df.shape[0] * sr)
best_threshold = sorted(y_train_pred_rf)[n-1]
print("Best threshold = {}".format(best_threshold))

Best threshold = 0.5091275515621155


In [34]:
success_rate_1 = []
for r in ['overall', 'white', 'afam', 'hisp', 'asian', 'other']:
    if r == 'overall':
        y_arr = np.array(y_test_pred_rf)
        y_actual = np.array(test_df.grad_6years)
    else:
        y_arr = np.array(y_test_pred_rf)[np.where(np.array(test_df[r]) == 1)[0]]
        y_actual = np.array(test_df.grad_6years)[np.where(np.array(test_df[r]) == 1)[0]]
    success_rate_1.append((r, len(y_arr), np.mean(y_actual), np.mean(y_arr > best_threshold)))
    fig = plt.figure(figsize=(16,11)) 
    ax = fig.add_subplot(1, 1, 1)
    pd.DataFrame({r:y_arr}).hist(r, bins = np.linspace(0,1,26), density=True, color='orange', figsize=(16,11), ax=ax)
    for p,q in zip(np.percentile(y_arr, q = [10,25,50,75,90]), [10,25,50,75,90]):
        ax.axvline(x=p, color='g', linestyle='dashed', linewidth=2)
        if r == "afam" and q == 10:
            ax.text(p-0.045,ax.get_ylim()[1]*1.005,"{}%".format(q),fontsize=20)
        elif r == "afam" and q == 25:
            ax.text(p-0.008,ax.get_ylim()[1]*1.005,"{}%".format(q),fontsize=20)
        else:
            ax.text(p-0.02,ax.get_ylim()[1]*1.005,"{}%".format(q),fontsize=20)
    ax.set_xlabel("Predicted Score", fontsize=32)
    ax.set_ylabel("Density", fontsize=32)
    if r != "afam":
        ax.set_title(r.capitalize(), fontsize=40, pad=40)
    else:
        ax.set_title("Black", fontsize=40, pad=40)
    # plt.show()
    plt.savefig(r +"_1.png")

In [35]:
race_column = []
for i in range(test_df.shape[0]):
    if test_df.white.iloc[i] == 1:
        race_column.append("white")
    elif test_df.afam.iloc[i] == 1:
        race_column.append("afam")
    elif test_df.hisp.iloc[i] == 1:
        race_column.append("hisp")
    elif test_df.asian.iloc[i] == 1:
        race_column.append("asian")
    elif test_df.other.iloc[i] == 1:
        race_column.append("other")
    else:
        race_column.append("mi")
race_column = np.array(race_column)
pred_y = np.array(y_test_pred_rf)
test_y = np.array(test_df.grad_6years)
pred_y = pred_y[race_column != "mi"]
test_y = test_y[race_column != "mi"]
race_column = race_column[race_column != "mi"]
print(len(race_column), len(pred_y), len(test_y))

62049 62049 62049


In [36]:
new_pred_real = pd.DataFrame({'pred_y': pred_y, 'real_y': test_y, 'race': race_column})
try:
    new_pred_real.loc[:,'pred_y_binned'] = pd.cut(new_pred_real.pred_y, bins=[0] + list(np.percentile(new_pred_real.pred_y, np.arange(2,100,2))) + [1])
except ValueError:
    new_pred_real.loc[:,'pred_y_binned'] = pd.cut(new_pred_real.pred_y.rank(method='first'), bins=[0] + list(np.percentile(new_pred_real.pred_y.rank(method='first'), np.arange(2,100,2))) + [new_pred_real.shape[0]+1])
try:
    new_pred_real.loc[:,'pred_y_binned_2'] = pd.cut(new_pred_real.pred_y, bins=[min(new_pred_real.pred_y) - 1e-3] + list(np.percentile(new_pred_real.pred_y, np.arange(10,100,10))) + [max(new_pred_real.pred_y) + 1e-3])
except ValueError:
    new_pred_real.loc[:,'pred_y_binned_2'] = pd.cut(new_pred_real.pred_y.rank(method='first'), bins=[0] + list(np.percentile(new_pred_real.pred_y.rank(method='first'), np.arange(10,100,10))) + [new_pred_real.shape[0]+1])
pct_dict = {e:(10*indx+5) for indx, e in enumerate(sorted(list(Counter(new_pred_real.pred_y_binned_2).keys())))}

In [56]:
new_pred_real.loc[:,'real_y'] = new_pred_real.real_y * 100
for r in ['afam']:
    print(r)
    new_sub = new_pred_real.copy()[new_pred_real.race.apply(lambda x: x in ['white', r])]
    new_sub = new_sub.groupby(['pred_y_binned', 'race']).agg({'real_y':'mean'}).reset_index()
    new_sub.loc[:,r] = new_sub.race.apply(lambda x: 1 if x == r else 0)
    new_sub = new_sub.sort_values([r, 'pred_y_binned'])
    print(new_sub.shape[0])
    new_sub.loc[:,'pred_score_percentile'] = list(np.linspace(1,99,50))*2
    new_sub = new_sub.rename(columns={'real_y':'share_of_actual_ABC'}).drop(['pred_y_binned'], axis=1)

    sns.set_style(style = "darkgrid")
    fig, ax = plt.subplots(1,1, figsize=(24,16.5))
    sns.scatterplot(x="pred_score_percentile", y="share_of_actual_ABC", hue='race', hue_order = ['white', r],
                    data=new_sub,
                    palette = ['C0','C2'], marker="x", ax=ax, s=150, alpha=0.7, linewidth = 3)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles=handles[1:], labels=['White', 'Black'], fontsize='40', markerscale=3)
    plt.xticks(np.linspace(0,100,11),fontsize=36)
    plt.yticks(np.linspace(0,100,11),fontsize=36)
    
    new_sub = new_pred_real.copy()[new_pred_real.race.apply(lambda x: x in ['white', r])]
    new_sub.loc[:,'pred_score_percentile_new'] = new_sub.pred_y_binned_2.apply(lambda x: pct_dict[x])
    np.random.seed(4321)
    sns.lineplot(data=new_sub, x="pred_score_percentile_new", y="real_y", hue='race', hue_order = ['white', r],
                 err_style="bars", err_kws = {'capsize': 8, 'elinewidth':3, 'capthick':2}, 
                 ci=95, ax=ax, linewidth = 6,
                 palette = ['C0','C2'], legend=False,
                 marker=".", markersize=36)
    plt.xlabel("Predicted Score Percentile", fontsize=40, labelpad=16)
    plt.ylabel("% of Students Who Earn Degree or Certificate in 6 Years", fontsize=36, labelpad=16)
    plt.savefig(r +"_predictive_parity.png")

afam
100
